<a href="https://colab.research.google.com/github/tkazusa/keras_practice/blob/master/Autopainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto painting with CAE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
import glob
import math
import random

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, ImageDataGenerator

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


### データの読み込み

In [0]:
data_path = 'PATH/TO/DIR'
data_lists = glob.glob(os.path.join(data_path, '*.jpg'))

val_n_sample = math.floor(len(data_lists)*0.1)
test_n_sample = math.floor(len(data_lists)*0.1)
train_n_sample = len(data_lists) - val_n_sample - test_n_sample

val_lists = data_lists[:val_n_sample]
test_lists = data_lists[val_n_sample:val_n_sample + test_n_sample]
train_lists = data_lists[val_n_sample + test_n_sample:train_n_sample + val_n_sample + test_n_sample]

In [0]:
print(len(train_lists), len(val_lists), len(test_lists))

23464 2932 2932


###  前処理：「RGB」を「LAB」に変換

In [0]:
import cv2


img_size = 224
def rgb2lab(rgb):
    assert rgb.dtype == 'uint8'
    return cv2.cvtColor(rgb, cv2.COLOR_RGB2Lab)

def lab2rgb(lab):
    assert lab.dtype == 'uint8'
    return cv2.cvtColor(lab, cv2.COLOR_Lab2RGB)

def get_lab_from_data_list(data_list):
    x_lab = []
    for f in data_list:
        rgb = img_to_array(
            load_img(
                f, 
                target_size=(img_size, img_size)
            )
        ).astype(np.uint8)
        lab = rgb2lab(rgb)
        x_lab.append(lab)
    return np.stack(x_lab)

### モデルの構築

In [0]:
from tensorflow.python.keras.layers import Conv2DTranspose

def build_autoencoder()
  autoencoder = Sequential()
  # Encoder
  autoencoder.add(Conv2D(32, (3, 3),  (1, 1),
                         activation='relu', padding='same',
                         input_shape=(224, 224, 1)))
  autoencoder.add(Conv2D(64, (3, 3), (2, 2), activation='relu', padding='same'))
  autoencoder.add(Conv2D(128, (3, 3), (2, 2), activation='relu', padding='same'))
  autoencoder.add(Conv2D(256, (3, 3), (2, 2), activation='relu', padding='same'))

  # Decoder
  autoencoder.add(Conv2DTranspose(128, (3, 3), (2, 2), activation='relu', padding='same'))
  autoencoder.add(Conv2DTranspose(64, (3, 3), (2, 2), activation='relu', padding='same'))
  autoencoder.add(Conv2DTranspose(32, (3, 3), (2, 2), activation='relu', padding='same'))
  autoencoder.add(Conv2D(2, (1, 1), (1, 1), activation='relu', padding='same'))

  autoencoder.compile(optimizer='adam', loss='mse')
  return autoencoder

autoencoder = build_autoencoder()
autoencoder.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 256)       295168    
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 56, 56, 128)       295040    
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 112, 112, 64)      73792     
_________________________________________________________________
con

### ジェネレータ関数の定義

In [0]:
def generator_with_preprocessing(data_list, batch_size, shuffle=False):
    while True:
        if shuffle:
            np.random.shuffle(data_list)
        for i in range(0, len(data_list), batch_size):
            batch_list = data_list[i:i + batch_size]
            batch_lab = get_lab_from_data_list(batch_list)
            batch_l = batch_lab[:, :, :, 0:1]
            batch_ab = batch_lab[:, :, :, 1:]
            yield (batch_l, batch_ab)
            
batch_size = 30

train_gen = generator_with_preprocessing(train_lists, batch_size, shuffle=True)
val_gen = generator_with_preprocessing(val_lists, batch_size)
test_gen = generator_with_preprocessing(test_lists, batch_size)

train_steps = math.ceil(len(train_lists)/batch_size)
val_steps = math.ceil(len(val_lists)/batch_size)
test_steps = math.ceil(len(test_lists)/batch_size)

### モデルの学習

In [0]:
from tensorflow.python.keras.callbacks import ModelCheckpoint

valid_ck = ModelCheckpoint('{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
callbacks = [valid_ck]

epochs= 150
    
autoencoder.fit_generator(
    generator=train_gen,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=val_gen,
    validation_steps=val_steps,
    # callbacks=callbacks, # コールバックを使用する場合はコメントアウトを外す
)

Epoch 1/150
783/783 [==============================]783/783 [==============================] - 354s 452ms/step - loss: 2423.3882 - val_loss: 1638.8180

Epoch 2/150
783/783 [==============================]783/783 [==============================] - 350s 447ms/step - loss: 457.8891 - val_loss: 266.4778

Epoch 3/150
783/783 [==============================]783/783 [==============================] - 350s 447ms/step - loss: 280.2091 - val_loss: 267.3231

Epoch 4/150
783/783 [==============================]783/783 [==============================] - 350s 447ms/step - loss: 277.8285 - val_loss: 309.7160

Epoch 5/150
783/783 [==============================]783/783 [==============================] - 348s 445ms/step - loss: 280.3955 - val_loss: 259.7945

Epoch 6/150
783/783 [==============================]783/783 [==============================] - 348s 445ms/step - loss: 279.2827 - val_loss: 258.8755

Epoch 7/150
783/783 [==============================]783/783 [==============================] - 349

783/783 [==============================]783/783 [==============================] - 343s 438ms/step - loss: 251.3499 - val_loss: 249.3952

Epoch 110/150
783/783 [==============================]783/783 [==============================] - 343s 438ms/step - loss: 250.2844 - val_loss: 246.8159

Epoch 111/150
783/783 [==============================]783/783 [==============================] - 343s 438ms/step - loss: 249.9835 - val_loss: 241.4105

Epoch 112/150
783/783 [==============================]783/783 [==============================] - 343s 438ms/step - loss: 250.9328 - val_loss: 238.3921

Epoch 113/150
783/783 [==============================]783/783 [==============================] - 343s 438ms/step - loss: 249.2512 - val_loss: 255.4767

Epoch 114/150
783/783 [==============================]783/783 [==============================] - 342s 437ms/step - loss: 250.3800 - val_loss: 242.5440

Epoch 115/150
783/783 [==============================]783/783 [==============================] - 342s 

### モデルの予測

In [0]:
preds = autoencoder.predict_generator(test_gen, steps=test_steps, verbose=0)

x_test = []
y_test = []
for i, (l, ab) in enumerate(generator_with_preprocessing(test_lists, batch_size)):
    x_test.append(l)
    y_test.append(ab)
    if i == (test_steps - 1):
        break
        
x_test = np.vstack(x_test)
y_test = np.vstack(y_test)

test_preds_lab = np.concatenate((x_test, preds), 3).astype(np.uint8)

test_preds_rgb = []
for i in range(test_preds_lab.shape[0]):
    preds_rgb = lab2rgb(test_preds_lab[i, :, :, :])
    test_preds_rgb.append(preds_rgb)
test_preds_rgb = np.stack(test_preds_rgb)

from IPython.display import display_png
from PIL import Image, ImageOps


for i in range(test_preds_rgb.shape[0]):
    gray_image = ImageOps.grayscale(array_to_img(test_preds_rgb[i]))
    display_png(gray_image)
    display_png(array_to_img(test_preds_rgb[i]))
    print('-'*25)
    if i == 20:
        break